<a href="https://colab.research.google.com/github/audio-bible/scripts/blob/master/Youtube_Downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Youtube Downloader

Download audio from [the KJV Audio Bible Channel on youtube](https://www.youtube.com/@skiesaboveus/playlists) of all bible chapters and save to google drive (My Drive/kjv_audio) sorted by chapters. Futher processing may be needed to make use of the audio.

You will likely exceed  google drive quota. So we suggest that once the download is done, you download the directory and do futher processing elsewhere



In [51]:
!pip install yt-dlp
#pip install -U https://github.com/coletdjnz/yt-dlp-youtube-oauth2/archive/refs/heads/master.zip # not needed in colab
!pip install youtube_dl

In [52]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
import yt_dlp
import os
import re

# Mount Google Drive
drive_path = '/content/drive/My Drive/kjv_audio'

# Ensure the top-level directory exists
os.makedirs(drive_path, exist_ok=True)
# Configuration
channel_url = 'https://www.youtube.com/@skiesaboveus/playlists'

def sanitize_title(title):
    # Remove trailing spaces and replace other spaces with underscores
    title = title.strip()
    title = re.sub(r'\s+', '_', title)  # Replace multiple spaces with single underscore
    return title

# def download_videos_from_playlist(playlist_url, download_dir):
#     ydl_opts = {
#         'quiet': False,
#         'format': 'bestaudio/best',  # Download the best audio quality
#         'extractaudio': True,  # Extract audio only
#         'audioformat': 'mp3',  # Desired audio format
#         'audioquality': '0',  # Best audio quality
#         'outtmpl': os.path.join(download_dir, '%(title)s.%(ext)s'),  # Save files in the given directory
#     }

#     with yt_dlp.YoutubeDL(ydl_opts) as ydl:
#         ydl.download([playlist_url])
def download_video(url, download_dir):
    def progress_hook(d):
        if d['status'] == 'error':
            print(f"Failed to download {url}: {d['error']}")
    ydl_opts = {
        'quiet': False,
        'format': 'bestaudio/best',  # Download the best audio quality
        'extractaudio': True,  # Extract audio only
        'audioformat': 'mp3',  # Desired audio format
        'audioquality': '0',  # Best audio quality
        'outtmpl': os.path.join(download_dir, '%(title)s.%(ext)s'),  # Save files in the given directory
        'noplaylist': True,  # Do not download the entire playlist
        'progress_hooks': [progress_hook],  # Add progress hooks for error handling
    }



    try:
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])
    except Exception as e:
        print(f"Failed to download {url}: {e}")

def get_expected_chapters_from_title(title):
    # Extract the number of chapters from the sanitized title
    match = re.search(r'Chapters_(\d{2})-(\d{2,3})', title)
    if match:
        start_chapter = int(match.group(1))
        end_chapter = int(match.group(2))
        return end_chapter - start_chapter + 1
    match = re.search(r'Chapter_(\d+)', title)
    if match:
        return 1
    return 0

def list_and_process_playlists(channel_url):
    ydl_opts = {
        'quiet': True,  # Suppress output to keep the script clean
        'extract_flat': True,  # Only get basic information, no downloads
        'force_generic_extractor': True,  # Handle generic extractors for playlist info
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(channel_url, download=False)

        # Extract playlists
        playlists = info_dict.get('entries', [])
        playlists.reverse()  # Reverse the list to get the most recent playlist first

        for playlist in playlists:
            title = playlist.get('title', 'No Title')
            url = playlist.get('url', 'No URL')

            # Check if title matches the pattern
            if re.match(r'\d{2} .+ Chapters \d{2}-\d{2,3} \(KJV\)', title) or re.match(r'\d{2} .+ Chapter \d+ \(KJV\)', title):
                # Sanitize title and create directory
                sanitized_title = sanitize_title(title.replace(' (KJV)', ''))
                directory_path = os.path.join(drive_path, sanitized_title)
                os.makedirs(directory_path, exist_ok=True)

                # Ensure existing files are not re-downloaded
                existing_files = set(os.listdir(directory_path))

                expected_chapters = get_expected_chapters_from_title(sanitized_title)

                if expected_chapters > 0:
                    # Check if the number of existing files matches the expected chapters
                    if len(existing_files) == expected_chapters:
                        print(f"Skipping Playlist Title: {title} (Already has {expected_chapters} chapters)")
                        print(f"Playlist URL: {url}")
                        print(f"Directory Path: {directory_path}")
                        print("-" * 40)
                        continue
                print(f"Expected: {expected_chapters}, but got {len(existing_files)}")
                # Download all videos in the playlist
                print(f"Processing Playlist Title: {title}")

                print(f"Playlist Title: {title}")
                print(f"Playlist URL: {url}")
                print(f"Directory Created: {directory_path}")
                # download_videos_from_playlist(url, directory_path)
                with yt_dlp.YoutubeDL({'quiet': True, 'extract_flat': True, 'force_generic_extractor': True}) as ydl:
                  info_dict = ydl.extract_info(url, download=False)
                  entries = info_dict.get('entries', [])
                  for entry in entries:
                      video_title = entry.get('title', 'No Title') + '.mp3'
                      video_url = entry.get('url', 'No URL')
                      if video_title not in existing_files:
                          download_video(video_url, directory_path)
                      else:
                          print(f"Skipping already existing video: {video_title}")
                print("-" * 40)
            else:
                print(f"Ignored Playlist Title: {title}")
                print(f"Playlist URL: {url}")
                print("-" * 40)

list_and_process_playlists(channel_url)


Expected: 28, but got 0
Processing Playlist Title: 40 Matthew Chapters 01-28 (KJV)
Playlist Title: 40 Matthew Chapters 01-28 (KJV)
Playlist URL: https://www.youtube.com/playlist?list=PL5389360540ED0D2A
Directory Created: /content/drive/My Drive/kjv_audio/40_Matthew_Chapters_01-28


[youtube] Extracting URL: https://www.youtube.com/watch?v=Hl2O2rQlhIY
[youtube] Hl2O2rQlhIY: Downloading webpage
[youtube] Hl2O2rQlhIY: Downloading ios player API JSON
[youtube] Hl2O2rQlhIY: Downloading web creator player API JSON
[youtube] Hl2O2rQlhIY: Downloading m3u8 information
[info] Hl2O2rQlhIY: Downloading 1 format(s): 251
[download] Destination: /content/drive/My Drive/kjv_audio/40_Matthew_Chapters_01-28/The Holy Bible - Matthew Chapter 1 (KJV).webm
[download] 100% of    2.73MiB in 00:00:00 at 4.95MiB/s   
[youtube] Extracting URL: https://www.youtube.com/watch?v=8trammXzd0A
[youtube] 8trammXzd0A: Downloading webpage
[youtube] 8trammXzd0A: Downloading ios player API JSON
[youtube] 8trammXzd0A: Downloading web creator player API JSON
[youtube] 8trammXzd0A: Downloading m3u8 information
[info] 8trammXzd0A: Downloading 1 format(s): 140
[download] Destination: /content/drive/My Drive/kjv_audio/40_Matthew_Chapters_01-28/The Holy Bible - Matthew Chapter 2 (KJV).m4a
[download] 100% of 

[youtube] Extracting URL: https://www.youtube.com/watch?v=y39MT83aEtQ
[youtube] y39MT83aEtQ: Downloading webpage
[youtube] y39MT83aEtQ: Downloading ios player API JSON
[youtube] y39MT83aEtQ: Downloading web creator player API JSON
[youtube] y39MT83aEtQ: Downloading m3u8 information
[info] y39MT83aEtQ: Downloading 1 format(s): 140
[download] Destination: /content/drive/My Drive/kjv_audio/41_Mark_Chapters_01-16/The Holy Bible - Mark Chapter 1 (KJV).m4a
[download] 100% of    4.57MiB in 00:00:00 at 6.91MiB/s   
[FixupM4a] Correcting container of "/content/drive/My Drive/kjv_audio/41_Mark_Chapters_01-16/The Holy Bible - Mark Chapter 1 (KJV).m4a"
[youtube] Extracting URL: https://www.youtube.com/watch?v=aeZqV_0TY04
[youtube] aeZqV_0TY04: Downloading webpage
[youtube] aeZqV_0TY04: Downloading ios player API JSON
[youtube] aeZqV_0TY04: Downloading web creator player API JSON
[youtube] aeZqV_0TY04: Downloading m3u8 information
[info] aeZqV_0TY04: Downloading 1 format(s): 140
[download] Destinat

[youtube] Extracting URL: https://www.youtube.com/watch?v=KaMypvoAOcQ
[youtube] KaMypvoAOcQ: Downloading webpage
[youtube] KaMypvoAOcQ: Downloading ios player API JSON
[youtube] KaMypvoAOcQ: Downloading web creator player API JSON
[youtube] KaMypvoAOcQ: Downloading m3u8 information
[info] KaMypvoAOcQ: Downloading 1 format(s): 251
[download] Destination: /content/drive/My Drive/kjv_audio/42_Luke_Chapters_01-24/The Holy Bible - Luke Chapter 1 (KJV).webm
[download] 100% of    6.95MiB in 00:00:01 at 5.47MiB/s   
[youtube] Extracting URL: https://www.youtube.com/watch?v=MIbjOwZOseM
[youtube] MIbjOwZOseM: Downloading webpage
[youtube] MIbjOwZOseM: Downloading ios player API JSON
[youtube] MIbjOwZOseM: Downloading web creator player API JSON
[youtube] MIbjOwZOseM: Downloading m3u8 information
[info] MIbjOwZOseM: Downloading 1 format(s): 140
[download] Destination: /content/drive/My Drive/kjv_audio/42_Luke_Chapters_01-24/The Holy Bible - Luke Chapter 2 (KJV).m4a
[download] 100% of    5.21MiB i

[youtube] Extracting URL: https://www.youtube.com/watch?v=Pe6eMQQIoqE
[youtube] Pe6eMQQIoqE: Downloading webpage
[youtube] Pe6eMQQIoqE: Downloading ios player API JSON
[youtube] Pe6eMQQIoqE: Downloading web creator player API JSON
[youtube] Pe6eMQQIoqE: Downloading m3u8 information
[info] Pe6eMQQIoqE: Downloading 1 format(s): 251
[download] Destination: /content/drive/My Drive/kjv_audio/43_John_Chapters_01-21/The Holy Bible - John Chapter 1 (KJV).webm
[download] 100% of    5.09MiB in 00:00:00 at 5.77MiB/s   
[youtube] Extracting URL: https://www.youtube.com/watch?v=2WOrcmnpBi0
[youtube] 2WOrcmnpBi0: Downloading webpage
[youtube] 2WOrcmnpBi0: Downloading ios player API JSON
[youtube] 2WOrcmnpBi0: Downloading web creator player API JSON
[youtube] 2WOrcmnpBi0: Downloading m3u8 information
[info] 2WOrcmnpBi0: Downloading 1 format(s): 140
[download] Destination: /content/drive/My Drive/kjv_audio/43_John_Chapters_01-21/The Holy Bible - John Chapter 2 (KJV).m4a
[download] 100% of    2.55MiB i

...

Failed to download https://www.youtube.com/watch?v=0wGrAnyPRe8: ERROR: [youtube] 0wGrAnyPRe8: Sign in to confirm you’re not a bot. This helps protect our community. Learn more
[youtube] Extracting URL: https://www.youtube.com/watch?v=FNOZOb7F67o
[youtube] FNOZOb7F67o: Downloading webpage
[youtube] FNOZOb7F67o: Downloading ios player API JSON
[youtube] FNOZOb7F67o: Downloading web creator player API JSON


ERROR: [youtube] FNOZOb7F67o: Sign in to confirm you’re not a bot. This helps protect our community. Learn more


Failed to download https://www.youtube.com/watch?v=FNOZOb7F67o: ERROR: [youtube] FNOZOb7F67o: Sign in to confirm you’re not a bot. This helps protect our community. Learn more
[youtube] Extracting URL: https://www.youtube.com/watch?v=aoWDN4xdrYg
[youtube] aoWDN4xdrYg: Downloading webpage
[youtube] aoWDN4xdrYg: Downloading ios player API JSON
[youtube] aoWDN4xdrYg: Downloading web creator player API JSON


ERROR: [youtube] aoWDN4xdrYg: Sign in to confirm you’re not a bot. This helps protect our community. Learn more


Failed to download https://www.youtube.com/watch?v=aoWDN4xdrYg: ERROR: [youtube] aoWDN4xdrYg: Sign in to confirm you’re not a bot. This helps protect our community. Learn more
----------------------------------------
Skipping Playlist Title: 03 Leviticus Chapters 01-27 (KJV) (Already has 27 chapters)
Playlist URL: https://www.youtube.com/playlist?list=PLEFB974CADB33A31D
Directory Path: /content/drive/My Drive/kjv_audio/03_Leviticus_Chapters_01-27
----------------------------------------
Skipping Playlist Title: 02 Exodus Chapters 01-40 (KJV) (Already has 40 chapters)
Playlist URL: https://www.youtube.com/playlist?list=PL61BF5C536664B991
Directory Path: /content/drive/My Drive/kjv_audio/02_Exodus_Chapters_01-40
----------------------------------------
Skipping Playlist Title: 01 Genesis Chapters 01-50 (KJV) (Already has 50 chapters)
Playlist URL: https://www.youtube.com/playlist?list=PL37B3F7739A3C6083
Directory Path: /content/drive/My Drive/kjv_audio/01_Genesis_Chapters_01-50
---------

In [36]:
%%bash
ls "/content/drive/My Drive/kjv_audio" | wc -l
ls "/content/drive/My Drive/kjv_audio"
ls "/content/drive/My Drive/kjv_audio/01_Genesis_Chapters_01-50"

66
01_Genesis_Chapters_01-50
02_Exodus_Chapters_01-40
03_Leviticus_Chapters_01-27
04_Numbers_Chapters_01-36
05_Deuteronomy_Chapters_01-34
06_Joshua_Chapters_01-24
07_Judges_Chapters_01-21
08_Ruth_Chapters_01-04
09_1_Samuel_Chapters_01-31
10_2_Samuel_Chapters_01-24
11_1_Kings_Chapters_01-22
12_2_Kings_Chapters_01-25
13_1_Chronicles_Chapters_01-29
14_2_Chronicles_Chapters_01-36
15_Ezra_Chapters_01-10
16_Nehemiah_Chapters_01-13
17_Esther_Chapters_01-10
18_Job_Chapters_01-42
19_Psalm_Chapters_01-150
20_Proverbs_Chapters_01-31
21_Ecclesiastes_Chapters_01-12
22_Song_of_Solomon_Chapters_01-08
23_Isaiah_Chapters_01-66
24_Jeremiah_Chapters_01-52
25_Lamentations_Chapters_01-05
26_Ezekiel_Chapters_01-48
27_Daniel_Chapters_01-12
28_Hosea_Chapters_01-14
29_Joel_Chapters_01-03
30_Amos_Chapters_01-09
31_Obadiah_Chapter_01
32_Jonah_Chapters_01-04
33_Micah_Chapters_01-07
34_Nahum_Chapters_01-03
35_Habakkuk_Chapters_01-03
36_Zephaniah_Chapters_01-03
37_Haggai_Chapters_01-02
38_Zechariah_Chapters_01-14
3

In [49]:
import os
from collections import defaultdict

def count_files_in_directories(base_dir):
    total_files = 0
    extension_count = defaultdict(int)  # Dictionary to store the count of each extension

    for root, dirs, files in os.walk(base_dir):
        total_files += len(files)
        for file in files:
            _, ext = os.path.splitext(file)
            if ext:  # Only count non-empty extensions
                extension_count[ext.lower()] += 1

    return total_files, extension_count

# Replace 'drive_path' with your actual directory path
drive_path = '/content/drive/My Drive/kjv_audio'

total_downloaded_files, extension_count = count_files_in_directories(drive_path)
not_downloaded_files = 1189 - total_downloaded_files

print(f"Total number of downloaded files: {total_downloaded_files}")
print(f"Total number of files to download: {not_downloaded_files}")

print("File extensions and their counts:")
for ext, count in sorted(extension_count.items()):
    print(f"{ext}: {count}")


Total number of downloaded files: 1187
Total number of files to download: 2
File extensions and their counts:
.m4a: 1130
.webm: 57


In [50]:
import os

def get_directory_size(directory):
    total_size = 0

    for dirpath, dirnames, filenames in os.walk(directory):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            # Add the size of the file to total_size
            total_size += os.path.getsize(filepath)

    return total_size

# Path to your directory
directory_path = '/content/drive/My Drive/kjv_audio'

# Get the total size of the directory
size_bytes = get_directory_size(directory_path)

# Convert bytes to a more readable format
def format_size(size_bytes):
    if size_bytes < 1024:
        return f"{size_bytes} bytes"
    elif size_bytes < 1024**2:
        return f"{size_bytes / 1024:.2f} KB"
    elif size_bytes < 1024**3:
        return f"{size_bytes / 1024**2:.2f} MB"
    else:
        return f"{size_bytes / 1024**3:.2f} GB"

print(f"Total size of '{directory_path}': {format_size(size_bytes)}")


Total size of '/content/drive/My Drive/kjv_audio': 3.64 GB


In [174]:
import os
import json
import re

# Define the path to the directories
base_path = '/content/drive/My Drive/kjv_audio'

def extract_book_names(base_path):
    book_names = {}

    # Regex patterns to handle different directory naming formats
    book_pattern = re.compile(r'^(\d{2})_(.*?)_Chapters?')
    chapter_pattern = re.compile(r'^(\d{2})_(.*?)_Chapter_(\d{1,2})')

    for dir_name in os.listdir(base_path):
        dir_path = os.path.join(base_path, dir_name)
        if os.path.isdir(dir_path):
            match = book_pattern.match(dir_name)
            if match:
                # Extract number and book name for "XX_Book_Chapters" format
                number = int(match.group(1))
                number_str = f"{number:02d}"  # Ensure two digits with leading zero
                book_name = match.group(2).replace('_', ' ').strip()

                # Add to dictionary
                book_names[number_str] = book_name
            else:
                match = chapter_pattern.match(dir_name)
                if match:
                    # Extract number and book name for "XX_Book_Chapter_X" format
                    number = int(match.group(1))
                    number_str = f"{number:02d}"  # Ensure two digits with leading zero
                    book_name = match.group(2).replace('_', ' ').strip()

                    # Add to dictionary
                    book_names[number_str] = book_name
                else:
                    # Print directory names that do not match any pattern
                    print(f"Unmatched directory: {dir_name}")

    return book_names

# Define the path for the JSON file
processed_path = '/content/drive/My Drive/kjv_audio_processed'
info_path = os.path.join(processed_path, 'info')
os.makedirs(info_path, exist_ok=True)
json_file_path = os.path.join(info_path, 'books_mapping.json')

# Extract book names and write to JSON file
book_names = extract_book_names(base_path)
book_names = dict(sorted(book_names.items()))
with open(json_file_path, 'w') as json_file:
    json.dump(book_names, json_file, indent=4)

print(f"Books mapping JSON file created at: {json_file_path}")


Books mapping JSON file created at: /content/drive/My Drive/kjv_audio_processed/info/books_mapping.json


In [175]:
%%bash
cat "/content/drive/My Drive/kjv_audio_processed/info/books_mapping.json" | wc -l

67


In [176]:
import json

# Define the path to the JSON file
json_file_path = '/content/drive/My Drive/kjv_audio_processed/info/books_mapping.json'

def check_all_numbers_in_keys(json_file_path, min_number, max_number):
    # Load JSON data from the file
    with open(json_file_path, 'r') as json_file:
        books_mapping = json.load(json_file)

    # Extract keys and convert them to integers
    keys = set(int(key) for key in books_mapping.keys())

    # Generate the required set of numbers
    required_numbers = set(range(min_number, max_number + 1))

    # Check for missing numbers
    missing_numbers = required_numbers - keys

    # Print results
    if not missing_numbers:
        print(f"All numbers from {min_number} to {max_number} are present as keys.")
    else:
        print(f"Missing numbers: {sorted(missing_numbers)}")

# Define the range
min_number = 1
max_number = 66

# Check if all numbers from 1 to 66 are present as keys
check_all_numbers_in_keys(json_file_path, min_number, max_number)



All numbers from 1 to 66 are present as keys.


In [177]:
import os
import json

# Define the path to the JSON file and the base path for creating directories
json_file_path = '/content/drive/My Drive/kjv_audio_processed/info/books_mapping.json'
base_dir = '/content/drive/My Drive/kjv_audio_processed/'

# Read the JSON file
with open(json_file_path, 'r') as json_file:
    book_names = json.load(json_file)

# Create directories for each book
for number, book_name in book_names.items():
    # Sanitize book name by replacing spaces with underscores
    sanitized_book_name = book_name.replace(' ', '_')
    # Define the full path for the new directory
    dir_path = os.path.join(base_dir, sanitized_book_name)
    # Create the directory
    os.makedirs(dir_path, exist_ok=True)
    # print(f"Directory created: {dir_path}")

print("All directories have been created.")


All directories have been created.


In [179]:
import os
import re
import json
import shutil

# Define the paths
base_path = '/content/drive/My Drive/kjv_audio'
processed_path = '/content/drive/My Drive/kjv_audio_processed'
info_path = os.path.join(processed_path, 'info')
books_mapping_file = os.path.join(info_path, 'books_mapping.json')

# Regex patterns
# book_pattern = re.compile(r'^(\d{2})_(.*?)_Chapters')
# chapter_pattern = re.compile(r'Chapter (\d+)')
book_pattern = re.compile(r'^(\d{2})_(.*?)_Chapters?')
chapter_pattern = re.compile(r'Chapter (\d{1,2})')


def read_books_mapping(json_file_path):
    with open(json_file_path, 'r') as json_file:
        return json.load(json_file)

def extract_book_and_chapters(base_path, books_mapping):
    book_info = {}

    for dir_name in os.listdir(base_path):
        dir_path = os.path.join(base_path, dir_name)
        if os.path.isdir(dir_path):
            # Extract book number and name
            book_match = book_pattern.match(dir_name)
            if book_match:
                book_number = book_match.group(1)
                book_name = book_match.group(2).replace('_', ' ').strip()
                book_info[book_number] = {'name': book_name, 'chapters': [], 'paths':[]}

                # print(f"Directory: {dir_name}")

                # Process files in the directory
                for file_name in os.listdir(dir_path):
                    if file_name.endswith('.m4a') or file_name.endswith('.webm'):
                        chapter_match = chapter_pattern.search(file_name)
                        if chapter_match:
                            # Extract the chapter number
                            chapter_number = int(chapter_match.group(1))
                            chapter_str = f"{chapter_number:03d}"
                            book_info[book_number]['chapters'].append(chapter_str)
                            book_info[book_number]['paths'].append(os.path.join(dir_path, file_name))

                            # print(f"File: {file_name} - Chapter: {chapter_str}")

                # Sort chapters and print
                book_info[book_number]['chapters'].sort()
                # print(f"Book Number: {book_number} - Book Name: {book_name}")
                # print(f"Chapters: {', '.join(book_info[book_number]['chapters'])}")
                # print("-" * 40)

    return book_info

def copy_and_rename_files(book_info, books_mapping, processed_path):
    file_counter = 0
    existing_files = 0
    for book_number, info in book_info.items():
        book_name = books_mapping.get(book_number, '')
        # print(f"Book Number: {book_number} - Book Name: {book_name}")
        if book_name:
            for chapter_index, chapter in enumerate(info['chapters']):
                source_file_path = info['paths'][chapter_index]
                source_file_name = os.path.basename(source_file_path)
                file_counter+=1
                # print(f"File: {source_file_name} - Chapter: {chapter}")

                if os.path.exists(source_file_path):
                    destination_dir = os.path.join(processed_path, book_name.replace(' ', '_'), chapter, 'kjv')
                    os.makedirs(destination_dir, exist_ok=True)

                    file_extension = os.path.splitext(source_file_name)[1]
                    destination_file_path = os.path.join(destination_dir, f"kjv-audio-bible{file_extension}")
                    # print(destination_file_path)

                    # skip if destination file exists
                    if not os.path.exists(destination_file_path):
                      shutil.copy2(source_file_path, destination_file_path)
                    else:
                      existing_files+=1
                      print(f"Skipping existing file: {destination_file_path} from {source_file_path}")
                      print(chapter)
                    # print(f"{source_file_path} => {destination_file_path}")
                else:
                    print(f"Source file not found: {source_file_path}")
        else:
            print(f"No book name found for book number: {book_number}")

    print(f"Total files copied: {file_counter}")
    print(f"Total existing files: {existing_files}")

# Read the books mapping from JSON
books_mapping = read_books_mapping(books_mapping_file)

# Extract book numbers and chapters
book_info = extract_book_and_chapters(base_path, books_mapping)
print(len(book_info))
# Copy and rename files
copy_and_rename_files(book_info, books_mapping, processed_path)


66
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/kjv/kjv-audio-bible.m4a
Skipping existing file: /content/drive/My Drive/kjv_audio_processed/Psalm/010/k

In [128]:
%%bash
ls '/content/drive/My Drive/kjv_audio_processed'|wc -l

67


In [167]:
import os

# Define the path to the base directory
base_path = '/content/drive/My Drive/kjv_audio_processed'

def count_files_in_directories(base_path, extensions):
    total_file_count = 0

    # Walk through all directories and files in the base path
    for root, dirs, files in os.walk(base_path):
        # Filter files based on the specified extensions
        filtered_files = [file for file in files if os.path.splitext(file)[1].lower() in extensions]
        num_files = len(filtered_files)
        total_file_count += num_files

    return total_file_count

# Define the extensions to look for
extensions = {'.m4a', '.webm'}

# Count files and print the total number
total_files = count_files_in_directories(base_path, extensions)
print(f"Total number of .m4a and .webm files in all directories: {total_files}")


Total number of .m4a and .webm files in all directories: 1138


In [170]:
%%bash
find '/content/drive/My Drive/kjv_audio_processed' \( -name '*.m4a' -o -name '*.webm' \) | wc -l


1138


In [180]:
%%bash
#rm -r '/content/drive/My Drive/kjv_audio_processed'